In [1]:
import xarray as xr
import numpy as np
from numpy import pi, sin, cos, arccos, clip, deg2rad
import numpy.ma as ma
from datetime import datetime
import dask
import time
import zarr

In [2]:
import gcsfs
gcs = gcsfs.GCSFileSystem()
import os
SCRATCH_BUCKET = os.environ['PANGEO_SCRATCH']

In [22]:
gcs.rm('pangeo-integration-te-3eea-prod-scratch-bucket/ofk123/dataset_alternative_3.zarr', recursive=True)
#gcs.ls('pangeo-integration-te-3eea-prod-scratch-bucket/ofk123/dataset_alternative_3.zarr')

In [23]:
n = 2105319
time = np.random.uniform(13000, 18500, n)
latitude = np.random.uniform(-80, 80, n) 
longitude = np.random.uniform(-180, 180, n)

#### Equally structured datasets stored in one zarr-store

### Storing data

In [24]:
ds1 = xr.Dataset(
    data_vars = dict(
        Anomalies = ( ["i", "pressure"], np.random.uniform(-1, 5, n).reshape(n,1) ),
        Mean = ( ["i", "pressure"], np.random.uniform(0, 2, n).reshape(n,1) ),
    ),
    coords=dict(
        i = (["i"], np.arange(n) ),
        time = ( ["i"], time),
        latitude = ( ["i"], latitude ),
        longitude = ( ["i"], longitude),
        pressure = (["pressure"], np.array([5]) ),
    ),
    attrs=dict(
        description = 'Description',
        pindex = 4,
        number_of_harmonics = 2,
        window_size = 500e3,
        creation_date = str( datetime.now() )
    ),
)


dsc = ds1.chunk()
path_to_zarrstore = f'{SCRATCH_BUCKET}/dataset_alternative_3.zarr'
mapper_alt3 = gcs.get_mapper(path_to_zarrstore)

%time dsc.to_zarr(mapper_alt3, consolidated=True)

CPU times: user 496 ms, sys: 225 ms, total: 722 ms
Wall time: 11.8 s


In [25]:
ds2 = xr.Dataset(
    data_vars = dict(
        Anomalies = ( ["i", "pressure"], np.random.uniform(-1, 5, n).reshape(n,1) ),
        Mean = ( ["i", "pressure"], np.random.uniform(0, 2, n).reshape(n,1) ),
    ),
    coords=dict(
        i = (["i"], np.arange(n) ),
        time = ( ["i"], time),
        latitude = ( ["i"], latitude ),
        longitude = ( ["i"], longitude),
        pressure = (["pressure"], np.array([10]) ),
    ),
    attrs=dict(
        description = 'Description',
        pindex = 4,
        number_of_harmonics = 2,
        window_size = 500e3,
        creation_date = str( datetime.now() )
    ),
)

dsc2 = ds2.chunk()
path_to_zarrstore = f'{SCRATCH_BUCKET}/dataset_alternative_3.zarr'
mapper_alt3 = gcs.get_mapper(path_to_zarrstore)
%time dsc2.to_zarr(mapper_alt3, consolidated=True, append_dim="pressure")

CPU times: user 518 ms, sys: 107 ms, total: 625 ms
Wall time: 11.1 s


In [26]:
ds3 = xr.Dataset(
    data_vars = dict(
        Anomalies = ( ["i", "pressure"], np.random.uniform(-1, 5, n).reshape(n,1) ),
        Mean = ( ["i", "pressure"], np.random.uniform(0, 2, n).reshape(n,1) ),
    ),
    coords=dict(
        i = (["i"], np.arange(n) ),
        time = ( ["i"], time),
        latitude = ( ["i"], latitude ),
        longitude = ( ["i"], longitude),
        pressure = (["pressure"], np.array([20]) ),
    ),
    attrs=dict(
        description = 'Description',
        pindex = 4,
        number_of_harmonics = 2,
        window_size = 500e3,
        creation_date = str( datetime.now() )
    ),
)

dsc3 = ds3.chunk()
path_to_zarrstore = f'{SCRATCH_BUCKET}/dataset_alternative_3.zarr'
mapper_alt3 = gcs.get_mapper(path_to_zarrstore)
%time dsc3.to_zarr(mapper_alt3, consolidated=True, append_dim="pressure")

CPU times: user 493 ms, sys: 109 ms, total: 602 ms
Wall time: 11.4 s


In [27]:
ds = xr.open_zarr(mapper_alt3, consolidated=True)
ds

<xarray.Dataset>
Dimensions:    (i: 2105319, pressure: 3)
Coordinates:
  * i          (i) int64 0 1 2 3 4 5 ... 2105314 2105315 2105316 2105317 2105318
    latitude   (i) float64 dask.array<chunksize=(2105319,), meta=np.ndarray>
    longitude  (i) float64 dask.array<chunksize=(2105319,), meta=np.ndarray>
  * pressure   (pressure) int64 5 10 20
    time       (i) float64 dask.array<chunksize=(2105319,), meta=np.ndarray>
Data variables:
    Anomalies  (i, pressure) float64 dask.array<chunksize=(2105319, 1), meta=np.ndarray>
    Mean       (i, pressure) float64 dask.array<chunksize=(2105319, 1), meta=np.ndarray>
Attributes:
    creation_date:        2023-08-02 12:23:33.346637
    description:          Description
    number_of_harmonics:  2
    pindex:               4
    window_size:          500000.0

In [28]:
xr.__version__, gcsfs.__version__, zarr.__version__,dask.__version__

('2022.12.0', '2022.11.0', '2.13.3', '2022.12.0')

In [29]:
gcs.ls('pangeo-integration-te-3eea-prod-scratch-bucket/ofk123/dataset_alternative_3.zarr')

['pangeo-integration-te-3eea-prod-scratch-bucket/ofk123/dataset_alternative_3.zarr/.zattrs',
 'pangeo-integration-te-3eea-prod-scratch-bucket/ofk123/dataset_alternative_3.zarr/.zgroup',
 'pangeo-integration-te-3eea-prod-scratch-bucket/ofk123/dataset_alternative_3.zarr/.zmetadata',
 'pangeo-integration-te-3eea-prod-scratch-bucket/ofk123/dataset_alternative_3.zarr/Anomalies',
 'pangeo-integration-te-3eea-prod-scratch-bucket/ofk123/dataset_alternative_3.zarr/Mean',
 'pangeo-integration-te-3eea-prod-scratch-bucket/ofk123/dataset_alternative_3.zarr/i',
 'pangeo-integration-te-3eea-prod-scratch-bucket/ofk123/dataset_alternative_3.zarr/latitude',
 'pangeo-integration-te-3eea-prod-scratch-bucket/ofk123/dataset_alternative_3.zarr/longitude',
 'pangeo-integration-te-3eea-prod-scratch-bucket/ofk123/dataset_alternative_3.zarr/pressure',
 'pangeo-integration-te-3eea-prod-scratch-bucket/ofk123/dataset_alternative_3.zarr/time']

##### Size of store, and one column, in megabytes

In [30]:
ds.nbytes/1e6, ds.Anomalies.sel(pressure=5).nbytes/1e6

(168.425544, 16.842552)

In [12]:
#ds.sortby("pressure").info

### Connect to a dask-cluster and set cluster-options

In [5]:
from dask_gateway import GatewayCluster, Gateway
from distributed import Client

g = Gateway()
g.list_clusters()

[]

In [6]:
options = g.cluster_options()
options.worker_cores = 2; options.worker_memory = 4
# Create a cluster with those options
cluster = g.new_cluster(options)

In [7]:
g.list_clusters()

[ClusterReport<name=prod.68beb2378684473a91204ca3b6211000, status=RUNNING>]

In [8]:
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /services/dask-gateway/clusters/prod.68beb2378684473a91204ca3b6211000/status,


In [10]:
cluster.scale(1)

### Loading of data from cloud-storage<br>
Want to load 4 out of 5 columns of data, into several dask-workers memory, and do embarrasingly parallel computations ( ~1e6  ).

In [12]:
@dask.delayed
def load_chunksauto(mapper, plevel):
    """Load data from zarr-store into worker-memory"""
    
    ds = xr.open_zarr(mapper, consolidated=True, chunks='auto') 
    data0 = ds.Anomalies.isel(pressure=plevel).load().values
    ii = ~xr.apply_ufunc(np.isnan, data0)
    data, time, lat, lon = data0[ii], ds.time.load().values[ii],ds.latitude.load().values[ii], ds.longitude.load().values[ii] 
    
    # Calculation using data-, lat-, lon-, and time-arrays
    #
    #
    #
    return None

In [13]:
@dask.delayed
def load_zarr(mapper, plevel):
    """Load data from zarr-store into worker-memory"""
    
    z = zarr.open_consolidated(mapper)
    data0 = z.Anomalies[:,plevel]
    ii = ~np.isnan(data0)
    data, time, lat, lon = data0[ii], z.time[:][ii], z.latitude[:][ii], z.longitude[:][ii]
    
    # Calculation using data-, lat-, lon-, and time-arrays
    #
    #
    #
    return None

In [14]:
@dask.delayed
def load_zarr_convenience(mapper, plevel): 
    """Load data from zarr-store into worker-memory"""
    
    z = zarr.convenience.open_consolidated(mapper)
    data0 = z.Anomalies[:,plevel]
    ii = ~np.isnan(data0)
    data, time, lat, lon = data0[ii], z.time[:][ii], z.latitude[:][ii], z.longitude[:][ii]
    
    # Calculation using data-, lat-, lon-, and time-arrays
    #
    #
    #
    return None

In [15]:
@dask.delayed
def load_scattered(data_scattered, time_scattered, lat_scattered, lon_scattered): 
    """Load data from zarr-store into worker-memory"""

    data, time, lat, lon = data_scattered, time_scattered, lat_scattered, lon_scattered
    # Calculation using data-, lat-, lon-, and time-arrays
    #
    #
    #
    return None

### Timing loading of data:<br>

In [31]:
plevel = 2
path_to_zarrstore = f'{SCRATCH_BUCKET}/dataset_alternative_3.zarr'
mapper_alt3 = gcs.get_mapper(path_to_zarrstore)

<br>
ds = xr.open_zarr(mapper, consolidated=True, chunks='auto'); ...  

In [90]:
%timeit -n 15 dask.compute( load_chunksauto(mapper_alt3, plevel) )[0]

2.24 s ± 40.4 ms per loop (mean ± std. dev. of 7 runs, 15 loops each)


<br>
ds = zarr.open_consolidated(mapper); ... 

In [88]:
%timeit -n 15 dask.compute( load_zarr(mapper_alt3, plevel) )[0]

801 ms ± 39.5 ms per loop (mean ± std. dev. of 7 runs, 15 loops each)


<br>
ds = zarr.convenience.open_consolidated(mapper); ... 

In [89]:
%timeit -n 15 dask.compute( load_zarr_convenience(mapper_alt3, plevel) )[0]

848 ms ± 250 ms per loop (mean ± std. dev. of 7 runs, 15 loops each)


<br>
distributing data to workers with client.scatter(data), before compute; ... 

In [32]:
ds = xr.open_zarr(mapper_alt3, consolidated=True, chunks='auto') 
data0 = ds.Anomalies.isel(pressure=plevel).load().values
ii = ~xr.apply_ufunc(np.isnan, data0)
data_scattered, time_scattered, lat_scattered, lon_scattered = client.scatter(data0[ii]), client.scatter(ds.time.load().values[ii]),client.scatter(ds.latitude.load().values[ii]), client.scatter(ds.longitude.load().values[ii]) 

In [33]:
%timeit -n 15 dask.compute( load_scattered(data_scattered, time_scattered, lat_scattered, lon_scattered) )[0]

18.2 ms ± 517 µs per loop (mean ± std. dev. of 7 runs, 15 loops each)


#### Scaling down and closing cluster

In [42]:
#cluster.scale(0)

In [64]:
cluster.close()

In [65]:
cluster.shutdown()